# **CS351-Lab 10**

# **Genetic Algorithm**
-The Three Palace Card Game

We have used genetic algorithm to simulate and evolve strategies for playing the card game The Three Palace. The goal is to train an agent (or population of agents) to make decisions based on the game's rules and maximize their performance.

In [ ]:
pip install torch numpy deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

# Define the Neural Network for playing the game
class PalaceCardGameNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(PalaceCardGameNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)  # To normalize outputs between 0 and 1 (probabilities)
        return x


In [ ]:
import random
from deap import base, creator, tools
import torch
import torch.nn as nn
import numpy as np

# Define the Neural Network for playing the game
class PalaceCardGameNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(PalaceCardGameNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)  # To normalize outputs between 0 and 1 (probabilities)
        return x

    def get_weights(self):
        """Return the parameters (weights) of the neural network as a flat tensor."""
        return torch.cat([param.view(-1) for param in self.parameters()])

    def set_weights(self, weights):
        """Set the parameters (weights) of the neural network from a flat tensor."""
        current_pos = 0
        for param in self.parameters():
            param_length = param.numel()
            param.data = weights[current_pos:current_pos+param_length].view(param.size())
            current_pos += param_length

# Define fitness and individual classes for DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize fitness
creator.create("Individual", object, fitness=creator.FitnessMax)

# Create an individual, which is a neural network object
def create_individual():
    net = PalaceCardGameNN(input_size=10, output_size=5)  # Example input/output sizes
    individual = creator.Individual([net])  # Store the neural network inside the individual
    individual.net = net  # Add a 'net' attribute to the individual for easier access
    return individual

# Define the fitness function
def evaluate(individual):
    net = individual.net  # Access the neural network using the 'net' attribute
    # Here, you would simulate the game to evaluate the performance of the NN
    # For simplicity, let's assume we have a function that plays the game
    fitness = simulate_game(net)
    return fitness,  # Return as a tuple (DEAP expects the result to be a tuple)

def simulate_game(net):
    # This function would run the game with the neural network (model)
    # For simplicity, simulate with a random fitness value
    return random.uniform(0, 1)  # Replace with actual game simulation

# Define the custom crossover function that operates on the neural network weights
def custom_crossover(ind1, ind2):
    # Get the neural network weights from both individuals
    net1_weights = ind1.net.get_weights()
    net2_weights = ind2.net.get_weights()

    # Perform a blend of the weights using a random gamma factor
    gamma = random.uniform(-0.5, 1.5)

    # Blend the weights of the neural networks
    new_weights1 = gamma * net1_weights + (1 - gamma) * net2_weights
    new_weights2 = gamma * net2_weights + (1 - gamma) * net1_weights

    # Set the new weights for each individual's neural network
    ind1.net.set_weights(new_weights1)
    ind2.net.set_weights(new_weights2)

# Define the custom mutation function that applies Gaussian noise to the weights
def custom_mutation(individual, mu=0, sigma=1, indpb=0.2):
    # Get the weights of the neural network
    weights = individual.net.get_weights()

    # Create Gaussian noise with the given mean (mu) and standard deviation (sigma)
    noise = torch.normal(mean=mu, std=sigma, size=weights.size())

    # Apply the noise to the weights (individual mutation)
    mutated_weights = weights + noise

    # Set the mutated weights back to the individual's neural network
    individual.net.set_weights(mutated_weights)

# Set up genetic algorithm components
toolbox = base.Toolbox()
toolbox.register("individual", create_individual)  # Directly use the create_individual function
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", custom_crossover)  # Use the custom crossover function
toolbox.register("mutate", custom_mutation)  # Use the custom mutation function
toolbox.register("select", tools.selTournament, tournsize=3)

# Run the genetic algorithm
def run_genetic_algorithm():
    population = toolbox.population(n=50)  # Population size
    ngen = 20  # Number of generations
    cxpb = 0.5  # Crossover probability
    mutpb = 0.2  # Mutation probability

    # Evaluation
    for gen in range(ngen):
        print(f"Generation {gen}")
        for ind in population:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # Select the best individuals
        selected = toolbox.select(population, len(population) // 2)

        # Clone the selected individuals
        offspring = list(map(toolbox.clone, selected))

        # Apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)  # Custom crossover
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)  # Custom mutation
                del mutant.fitness.values

        # Recalculate fitness values
        for ind in offspring:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # Replace old population with new one
        population[:] = offspring

    return population

# Run the GA to evolve neural networks
if __name__ == "__main__":
    final_population = run_genetic_algorithm()
    print("Final population evolved.")


Generation 0
Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Final population evolved.


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
import random
import pandas as pd

# Define constants
SUITS = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

# Create a deck of cards
def create_deck():
    return [f'{rank} of {suit}' for suit in SUITS for rank in RANKS]

# Shuffle deck
def shuffle_deck(deck):
    random.shuffle(deck)

# Simulate a game step
def simulate_game_step(player_hand, discard_pile, action, deck):
    """Simulate a game step by performing an action."""

    if action == 'play':
        if player_hand:  # Check if player has cards to play
            card_played = random.choice(player_hand)
            player_hand.remove(card_played)
            discard_pile.append(card_played)
        else:
            return player_hand, discard_pile, deck, 'pass'  # If no cards to play, pass

    elif action == 'draw':
        if deck:  # Check if there are cards left to draw
            card_drawn = deck.pop()
            player_hand.append(card_drawn)
        else:
            return player_hand, discard_pile, deck, 'pass'  # If deck is empty, pass

    elif action == 'pass':
        pass  # Do nothing

    return player_hand, discard_pile, deck, action

# Function to simulate a complete game and generate a dataset
def simulate_game(player_count=4, game_length=20):
    deck = create_deck()
    shuffle_deck(deck)

    # Initialize hands for players
    hands = {f'Player_{i}': [deck.pop() for _ in range(5)] for i in range(player_count)}
    discard_pile = []

    data = []

    for turn in range(game_length):
        for player, hand in hands.items():
            # Generate features
            state = {
                'turn': turn,
                'player_hand': hand,
                'discard_pile': discard_pile,
                'opponent_hands': [hands[opp] for opp in hands if opp != player],
                'remaining_deck': len(deck)
            }

            # Simulate an action (for simplicity, choose randomly)
            action = random.choice(['play', 'draw', 'pass'])

            # Apply the action to the game state
            new_hand, new_discard_pile, new_deck, applied_action = simulate_game_step(hand, discard_pile, action, deck)

            # Append data
            data.append({
                'turn': turn,
                'player': player,
                'player_hand': ' '.join(new_hand),
                'discard_pile': ' '.join(new_discard_pile),
                'action': applied_action,
                'remaining_deck': len(new_deck)
            })

            # Update the hands and deck
            hands[player] = new_hand
            discard_pile = new_discard_pile
            deck = new_deck

    return pd.DataFrame(data)

# Generate a dataset by simulating 100 games
def generate_dataset(num_games=100):
    all_data = []
    for _ in range(num_games):
        game_data = simulate_game()
        all_data.append(game_data)

    return pd.concat(all_data, ignore_index=True)

# Simulate and save the dataset
df = generate_dataset(num_games=10)  # Simulate 10 games for testing
df.to_csv('palace_card_game_dataset.csv', index=False)

# Display the first few rows of the dataset
print(df.head())


   turn    player                                        player_hand  \
0     0  Player_0    A of Clubs 5 of Clubs 6 of Hearts 7 of Diamonds   
1     0  Player_1  10 of Hearts 3 of Hearts 5 of Spades 9 of Club...   
2     0  Player_2   7 of Clubs Q of Spades K of Diamonds 10 of Clubs   
3     0  Player_3  9 of Hearts 8 of Hearts J of Spades A of Heart...   
4     1  Player_0    A of Clubs 5 of Clubs 6 of Hearts 7 of Diamonds   

                 discard_pile action  remaining_deck  
0                 8 of Spades   play              32  
1                 8 of Spades   draw              31  
2  8 of Spades 10 of Diamonds   play              31  
3  8 of Spades 10 of Diamonds   draw              30  
4  8 of Spades 10 of Diamonds   pass              30  
